In [5]:
# Importing Required Library
import pandas as pd
import lightgbm as lgb
import lightgbm
# Similarly LGBMRegressor can also be imported for a regression model.
from lightgbm import LGBMClassifier

In [6]:
# Reading the train and test dataset
data = pd.read_csv("/Users/brocktbennett/GitHub/Project Data/2023_TAMU_competition_data/Merged/New/cleaned_humana_inner9-29.csv")

/var/folders/_d/gcvc5_md1bg3809rkn2_lqk40000gn/T/ipykernel_1349/3525296596.py:2: DtypeWarning: Columns (11,12,13,14,17,87,89,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/Users/brocktbennett/GitHub/Project Data/2023_TAMU_competition_data/Merged/New/cleaned_humana_inner9-29.csv")


In [7]:
data.head()

,Unnamed: 0,therapy_id,medclm_key,clm_unique_key,primary_diag_cd,visit_date,Visit_Duration,diag_cd2,diag_cd3,diag_cd4,...,id,therapy_start_date,therapy_end_date,Date Duration,tgt_ade_dc_ind,race_cd,est_age,sex_cd,cms_disabled_ind,cms_low_income_ind
0,0,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
1,1,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
2,2,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
3,3,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
4,4,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No


In [8]:
# Removing Columns not Required
data = data.drop(columns = ['Unnamed: 0'], axis = 1)
data.head()

,therapy_id,medclm_key,clm_unique_key,primary_diag_cd,visit_date,Visit_Duration,diag_cd2,diag_cd3,diag_cd4,diag_cd5,...,id,therapy_start_date,therapy_end_date,Date Duration,tgt_ade_dc_ind,race_cd,est_age,sex_cd,cms_disabled_ind,cms_low_income_ind
0,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,Z7982,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
1,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,Z7982,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
2,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,Z7982,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
3,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,Z7982,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
4,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,Z7982,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No


In [9]:
#consolidating dataframe into necessary columns...
#pd.set_option('display.max_columns', None)
data.head()
model_data = data[['therapy_id', 'Visit_Duration', 'MedProcess_Duration', 'pot', 'util_cat', 'hedis_pot', 'clm_type_x', 
                   'ade_diagnosis', 'seizure_diagnosis', 'pain_diagnosis', 'fatigue_diagnosis', 'nausea_diagnosis', 
                   'hyperglycemia_diagnosis', 'constipation_diagnosis', 'diarrhea_diagnosis','PD_C00ÐD49_FLG', 'PD_D50ÐD89_FLG',	
                   'PD_E00ÐE90_FLG',	'PD_G00ÐG99_FLG',	'PD_I00ÐI99_FLG',	'PD_J00ÐJ99_FLG',	'PD_M00ÐM99_FLG',	'PD_R00ÐR99_FLG',	
                   'PD_Z00ÐZ99_FLG',	'PD_Other_FLG',	'NPD_C00ÐD49',	'NPD_D50ÐD89', 'NPD_E00ÐE90',	'NPD_G00ÐG99',	'NPD_I00ÐI99',	
                   'NPD_J00ÐJ99',	'NPD_K00ÐK93',	'NPD_M00ÐM99',	'NPD_R00ÐR99',	'NPD_Z00ÐZ99',	'NPD_Other',	'NPD_SUM', 
                   'Prescription_Filled_Duration', 'RX_Process_Duration', 'pay_day_supply_cnt', 'rx_cost', 'tot_drug_cost_accum_amt', 
                   'mail_order_iNod', 'generic_ind', 'maint_ind', 'metric_strength', 'clm_type_y', 'ddi_ind', 'anticoag_ind',	
                   'diarrhea_treat_ind',	'nausea_treat_ind',	'seizure_treat_ind',	'RxGroupings', 'Date Duration', 'tgt_ade_dc_ind', 
                   'race_cd', 'est_age', 'sex_cd', 'cms_disabled_ind', 'cms_low_income_ind']] 

In [10]:
model_data.head()

,therapy_id,Visit_Duration,MedProcess_Duration,pot,util_cat,hedis_pot,clm_type_x,ade_diagnosis,seizure_diagnosis,pain_diagnosis,...,nausea_treat_ind,seizure_treat_ind,RxGroupings,Date Duration,tgt_ade_dc_ind,race_cd,est_age,sex_cd,cms_disabled_ind,cms_low_income_ind
0,1066310426-TAGRISSO-1,-128.0,72,Outpatient,Outpatient,Other,med,No,No,No,...,No,No,Infection,30,0,White,74.0,M,No,No
1,1066310426-TAGRISSO-1,-128.0,72,Outpatient,Outpatient,Other,med,No,No,No,...,No,No,Infection,30,0,White,74.0,M,No,No
2,1066310426-TAGRISSO-1,-128.0,72,Outpatient,Outpatient,Other,med,No,No,No,...,No,No,Respiratory,30,0,White,74.0,M,No,No
3,1066310426-TAGRISSO-1,-128.0,72,Outpatient,Outpatient,Other,med,No,No,No,...,No,No,Cardio,30,0,White,74.0,M,No,No
4,1066310426-TAGRISSO-1,-128.0,72,Outpatient,Outpatient,Other,med,No,No,No,...,No,No,Respiratory,30,0,White,74.0,M,No,No


In [13]:
# Convert categorical variables to dummy variables or map to numeric values
race = pd.get_dummies(model_data['race_cd'], dtype=int, drop_first=False)
ade_flag = pd.get_dummies(model_data['ade_diagnosis'], dtype=int, drop_first=True)
seizure_flag = pd.get_dummies(model_data['seizure_diagnosis'], dtype=int, drop_first=True)
pain_flag = pd.get_dummies(model_data['pain_diagnosis'], dtype=int, drop_first=True)
mail_flag = pd.get_dummies(model_data['mail_order_iNod'], dtype=int, drop_first=True)

# Map 'generic_ind' and 'maint_id' to binary values
generic_flag = model_data['generic_ind'].replace({'GENERIC': 1, 'BRANDED': 0})
maintenance_flag = model_data['maint_ind'].replace({'MAINT': 1, 'NONMAINT': 0})

# Convert 'ddi_ind' to dummy variables
ddi_flag = pd.get_dummies(model_data['ddi_ind'], dtype=int, drop_first=True)

# Convert 'tgt_ade_dc_ind' from Yes and No to 1 and 0 (Binary)
data['tgt_ade_dc_ind'] = data['tgt_ade_dc_ind'].replace({'Yes': 1, 'No': 0})

# INSERT HOLDOUT DATA HERE

In [23]:
# import pandas as pd
# 
# # Define file paths for your CSV files
# med_holdout_path = "/Users/brocktbennett/GitHub/Project Data/2023_TAMU_competition_data/Cleaned Holdout/medclms_holdout_cleaned9_29.csv"
# rx_holdout_path = "/Users/brocktbennett/GitHub/Project Data/2023_TAMU_competition_data/Cleaned Holdout/rxclms_holdout_cleaned9_29.csv"
# 
# # Read holdout data from CSV files with specified encoding
# try:
#     med_holdout = pd.read_csv(med_holdout_path, encoding='utf-8')
# except UnicodeDecodeError:
#     med_holdout = pd.read_csv(med_holdout_path, encoding='latin1')
# 
# try:
#     rx_holdout = pd.read_csv(rx_holdout_path, encoding='utf-8')
# except UnicodeDecodeError:
#     rx_holdout = pd.read_csv(rx_holdout_path, encoding='latin1')



In [24]:
# Skipping Data Exploration
# Dummification of Diagnosis Column --should we use 
dummies = pd.get_dummies(data['race_cd'])

# Add the dummies to the original data frame 
data = pd.concat([data, dummies], axis=1)

In [25]:
# Splitting Dataset in two parts for testing
train = model_data[0:400]
test = model_data[400:568]

#FOR REAL
###test = pd.read_csv("HOLDOUT_MERGED_.csv")


In [26]:
# Separating the independent and target variable on both data set
x_train = train.drop(columns =['tgt_ade_dc_ind'], axis = 1)
y_train = train['tgt_ade_dc_ind']
x_test = test.drop(columns =['tgt_ade_dc_ind'], axis = 1)
y_test = test['tgt_ade_dc_ind']


In [27]:
# Creating an object for model and fitting it on training data set
model = LGBMClassifier()
model.fit(x_train, y_train)

# Predicting the Target variable
pred = model.predict(x_test)
print(pred)
accuracy = model.score(x_test, y_test)
print(accuracy)



ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: therapy_id: object, pot: object, util_cat: object, hedis_pot: object, clm_type_x: object, ade_diagnosis: object, seizure_diagnosis: object, pain_diagnosis: object, fatigue_diagnosis: object, nausea_diagnosis: object, hyperglycemia_diagnosis: object, constipation_diagnosis: object, diarrhea_diagnosis: object, mail_order_iNod: object, generic_ind: object, maint_ind: object, clm_type_y: object, ddi_ind: object, anticoag_ind: object, diarrhea_treat_ind: object, nausea_treat_ind: object, seizure_treat_ind: object, RxGroupings: object, race_cd: object, sex_cd: object, cms_disabled_ind: object, cms_low_income_ind: object